# Electronic Structure Calculations using Generalized Ising Hamiltonians

Your final task is to calculate the ground state energy of a hydrogen molecule using a Generalized Ising Hamiltonian:

$$ 
H = E_0 + \sum_i h_i\sigma_i + \sum_{ij} J_{ij}\sigma_i\sigma_j
+ \sum_{ijk} K_{ijk}\sigma_i\sigma_j\sigma_k
+ \sum_{ijkl} L_{ijkl}\sigma_i\sigma_j\sigma_k\sigma_l
+ \cdots
$$

where the Hamiltonian parameters ($E_0, h_i, J_{ij}, K_{ijk}, L_{ijkl}$) will be provided to you by the `read_generalized_ising_hamiltonian` function below.

These Ising Hamiltonians were produced using the *Iterative Qubit Coupled Cluster* method (https://arxiv.org/abs/1906.11192).

The $H_2$ Ising Hamiltonian only needs 4 spins to fully describe the ground state, hence our Hamiltonian only needs a 4-point interaction at most.

In [1]:
import numpy as np

from abstract_ising import AbstractIsing
from ising_animator import IsingAnimator
import copy
%matplotlib inline

In [2]:
def read_generalized_ising_hamiltonian(path):
    with open(path, "r") as f:
        f.readline()  # discard first line
        compressed_hamiltonian = [
            tuple(line.strip().split())
            for line in f.readlines()
        ]
    
    num_sites = len(compressed_hamiltonian[0][0])
    hamiltonian_terms = [np.zeros((num_sites,)*i) for i in range(num_sites+1)]

    for sites, val in compressed_hamiltonian:
        num_zs = 0
        site_nums = []
        for i, x in enumerate(sites):
            if x == 'z':
                site_nums.append(i)
                num_zs += 1

        hamiltonian_terms[num_zs][tuple(site_nums)] = float(val)

    return hamiltonian_terms

In [3]:
E0, h, J, K, L = read_generalized_ising_hamiltonian("./hamiltonians/Ising-H2-STO-3G-bk-samespin-R=0.65.inp")

In [4]:
a = np.arange(25).reshape(5,5)
b = np.arange(5)
c = np.arange(6).reshape(2,3)
print(a)
print(np.einsum('ii', a))
print("---")
print(np.einsum('ii->i', a))
print("---")
print(np.einsum('ij', a))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
60
---
[ 0  6 12 18 24]
---
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


First, you must write a class that describes the Generalized Ising Model given the arrays containing the Hamiltonian parameters. You will need to write a function which computes energy of the stored spin configuration (see the previous tasks for inspiration), as well as a function which computes the change in energy resulting from a single-spin-flip (you could of course use the naive approach and simply compute the energy of two spin configurations and subtract one from the other).

In [5]:
# Write a class to describe the generalized Ising Model given its Hamiltonian parameters

class GeneralizedIsingModel(AbstractIsing):
    def __init__(self,E0,h,J,K,L,N):
        self.E0  = E0
        self.h, self.J = h , J
        self.K, self.L = K, L
        self.N = N
        self.num_spins = self.N
        
        # initialize system at infinite temperature
        # i.e. spins are completely random and uncorrelated
        self.spins = 2*(np.random.rand(self.N) < 0.5) - 1
    
    def energy(self):
        """Returns the energy of the current spin configuration"""
        tot_h = 0.0
        tot_J = 0.0
        tot_K = 0.0
        tot_L = 0.0
        for i in range(0,self.N):
            tot_h += self.h[i]*self.spins[i]
            for j in range(0,self.N):
                tot_J += (self.J[i,j])*(self.spins[i]*self.spins[j])
                for k in range(0,self.N):
                    tot_K += (self.K[i,j,k])*(self.spins[i]*self.spins[j]*self.spins[k])
                    for l in range(0,self.N):
                        tot_L += (self.L[i,j,k,l])*(self.spins[i]*self.spins[j]*self.spins[k]*self.spins[l])
        
        return self.E0 + tot_h + tot_J + tot_K + tot_L
        
    
    def energy_diff(self, i):
        #temp_spins = copy.deepcopy(self.spins)
        e1 = self.energy()
        
        #Flip i temporarily 
        self.spins[i] = (-1)*self.spins[i]
        
        e2 = self.energy()
        
        #Revert
        self.spins[i] = (-1)*self.spins[i]
        
        return e2-e1
    
    def rand_site(self):
        return (np.random.randint(self.N),)

In [6]:
np.random.seed(4)
ising = GeneralizedIsingModel(E0,h,J,K,L,4)
ising.energy()
print(ising.spins)
print(ising.energy_diff(0))
print(ising.spins)

[-1 -1 -1 -1]
-0.7967113953728346
[-1 -1 -1 -1]


Next you'll run a Monte Carlo simulation for this model at some finite temperature for 1000 steps, printing out the energy of the state every so often

In [7]:
# your MC simulation here
np.random.seed(4)
ising = GeneralizedIsingModel(E0,h,J,K,L,4)

for t in range(1000):
    E = ising.mc_step(T=1.0)
    
    if t % 50 == 0:
        print(E)


-0.4855710768185992
0.5724567781741282
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-0.19222515402425988
-1.1299047752322906
-0.4020519317130593
-0.3331933798594561
-1.1299047752322906
-0.4020519317130593
-0.333193379859456
-1.1299047752322906
-0.19222515402425988
-0.25924181243352884
-0.333193379859456
-0.33319337985945596
-0.48557107681859907
-0.3331933798594562
-0.19222515402425988


Now, apply (one of) the annealing procedure(s) you came up with in the previous task to this problem to find a ground state of the system:

In [12]:
# your annealing code here
N = 2000
t = np.arange(N)
T_i = 100
T_f = 0.01

#Exponential works the best and is default for many systems, lets try with that
T_exp = T_i * ((T_f/T_i) ** (t/N))

#Reinitialize
np.random.seed(4)
ising = GeneralizedIsingModel(E0,h,J,K,L,4)
for t in range(1000):
    E = ising.mc_step(T=T_exp[t])
    
    if t % 50 == 0:
        print(E)

-0.4855710768185992
-0.25924181243352884
-0.19222515402425988
1.144680802606227
0.39731238040938205
0.7256454984385436
-0.4855710768185992
0.5724567781741281
-0.40205193171305953
-0.48557107681859907
-1.1299047752322906
-1.1299047752322906
-0.25924181243352884
-0.48557107681859907
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906
-1.1299047752322906


Finally, iterate over the entire spin configuration space (this is tractable since we only have 4 spins) to find the exact ground state energy. Compare this energy to the one you found above using your annealer.

In [9]:
# exact ground state calculation
#Consider all the permutations of the configurations of spins
print("----Exhaustive search------")
dim = np.arange(2 ** ising.num_spins)
print(dim)
space = ((dim[:, None] & (1 << np.arange(ising.num_spins))) > 0)
#print(space)
space = 2*space.astype(int) - 1
min = None
for i in range(0,2**ising.num_spins):
    ising.spins = space[i]
    print("Spins: ",space[i])
    temp_energy = ising.energy()
    print("Energy: ",temp_energy)
    if min == None:
        min = temp_energy
    elif min > temp_energy:
        min = temp_energy
print("The ground state is: ",min)

----Exhaustive search------
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Spins:  [-1 -1 -1 -1]
Energy:  -0.333193379859456
Spins:  [ 1 -1 -1 -1]
Energy:  -1.1299047752322906
Spins:  [-1  1 -1 -1]
Energy:  -0.48557107681859907
Spins:  [ 1  1 -1 -1]
Energy:  -0.25924181243352884
Spins:  [-1 -1  1 -1]
Energy:  -0.33319337985945613
Spins:  [ 1 -1  1 -1]
Energy:  1.144680802606227
Spins:  [-1  1  1 -1]
Energy:  0.5724567781741281
Spins:  [ 1  1  1 -1]
Energy:  -0.40205193171305953
Spins:  [-1 -1 -1  1]
Energy:  0.5724567781741281
Spins:  [ 1 -1 -1  1]
Energy:  -0.19222515402425988
Spins:  [-1  1 -1  1]
Energy:  0.39731238040938205
Spins:  [ 1  1 -1  1]
Energy:  0.7256454984385436
Spins:  [-1 -1  1  1]
Energy:  -0.4855710768185992
Spins:  [ 1 -1  1  1]
Energy:  -0.40205193171305936
Spins:  [-1  1  1  1]
Energy:  0.39731238040938194
Spins:  [1 1 1 1]
Energy:  0.8171587078823377
The ground state is:  -1.1299047752322906


Now, clean up your code a little, and write a for-loop that iterates over all the different values of the Hydrogen seperation distance $R$ available in the `hamiltonians` directory. 

For each $R$ you must:
- Read in the associated Ising Hamiltonian
- Perform an annealed Monte Carlo simulation to find a candidate ground state energy
- Compute the exact ground state energy
- Compare the two results

In [17]:
filestr = "Ising-H2-STO-3G-bk-samespin-R="
r_0 = 0.65
cur_r = r_0

#Exponential decay increased steps
N = 2000
t = np.arange(N)
T_i = 100
T_f = 0.01

T_exp = T_i * ((T_f/T_i) ** (t/N))

for i in range(0,20):
    filename = "./hamiltonians/" + filestr + str(round(cur_r,2)) + ".inp"
    print("-----R="+str(round(cur_r,2))+"-----")
    E0, h, J, K, L = read_generalized_ising_hamiltonian(filename)
    
    #Initialize
    np.random.seed(4)
    ising = GeneralizedIsingModel(E0,h,J,K,L,4)
    
    #MC procedure
    for t in range(2000):
        E = ising.mc_step(T=T_exp[t])
    
    mc_energy = ising.energy()
    
    #Brute force
    min = None
    for i in range(0,2**ising.num_spins):
        ising.spins = space[i]
        temp_energy = ising.energy()
        if min == None:
            min = temp_energy
        elif min > temp_energy:
            min = temp_energy
    
    print("MC State Energy: ",mc_energy)
    print("Ground State Energy: ",min)
    if min == mc_energy:
        print("Annealing could find the ground state")
    else:
        print("Annealing couldn't find the ground state")
    
    cur_r += 0.15

-----R=0.65-----
MC State Energy:  -1.1299047752322906
Ground State Energy:  -1.1299047752322906
Annealing could find the ground state
-----R=0.8-----
MC State Energy:  -1.134147672223387
Ground State Energy:  -1.134147672223387
Annealing could find the ground state
-----R=0.95-----
MC State Energy:  -1.1113394317141148
Ground State Energy:  -1.1113394317141148
Annealing could find the ground state
-----R=1.1-----
MC State Energy:  -1.0791929635915072
Ground State Energy:  -1.0791929635915072
Annealing could find the ground state
-----R=1.25-----
MC State Energy:  -0.8427811750184449
Ground State Energy:  -1.0457831649744005
Annealing couldn't find the ground state
-----R=1.4-----
MC State Energy:  -1.0154682691531116
Ground State Energy:  -1.0154682691531116
Annealing could find the ground state
-----R=1.55-----
MC State Energy:  -0.9904763585526107
Ground State Energy:  -0.9904763585526107
Annealing could find the ground state
-----R=1.7-----
MC State Energy:  -0.9714267029717627
Gro

For many of the <b>R</b> values where Annealing couldn't find solution (eg R=2.75), the final energies are pretty close to the ground state. A longer anneal schedule can hopefully help in fixing some of them. 